In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.isnull().sum() # Check if there are missing values

In [ ]:
# I'm going to OneHotEncode the categorical columns with the get.dummies function

encoded_dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], dtype=int)
encoded_dataset
dataset = pd.concat([dataset, encoded_dataset], axis=1)
dataset.drop('sex', axis=1, inplace=True)
dataset.drop('smoker', axis=1, inplace=True)
dataset.drop('region', axis=1, inplace=True)
dataset

In [ ]:
train_cols = dataset.columns.drop('expenses')
X = dataset[train_cols].values
y = dataset['expenses'].values

In [ ]:
from sklearn.model_selection import train_test_split
# With train_test_split method split the dataset into 80% train data and 20% test data
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, train_size=0.8)

In [ ]:
from keras import callbacks
# Model creation
model = keras.Sequential([
        layers.Dense(units=64, input_dim=X.shape[1], activation = 'relu' ),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
        ])

# Compile the model using adam and mae
model.compile(
    optimizer='adam',
    loss ='mae',
    metrics=['mean_absolute_error', 'mean_squared_error']
)

early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, restore_best_weights=True)

model.fit(train_dataset, train_labels,
          validation_data = (test_dataset, test_labels),
          epochs = 1000,
          callbacks = [early_stopping],
          verbose = 2
          )

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
# For some reason the plot failed so i commented it
'''test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)'''